In [2]:
import sys
import time
import pandas as pd
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json
import csv
import ast

In [3]:
CONFLICT_PROMPT_FLOODLIGHT = """
You are an expert network engineer analyzing two Floodlight (OpenFlow) flow rules to decide whether they conflict.

PRIMARY DECISION
A conflict exists if BOTH are true:
1) The rules apply to the same switch (equal "switch"; if both specify "table", tables must be equal too).
2) Their match conditions overlap (there exists at least one packet that matches both),
   AND either:
   a) their actions differ, or
   b) they are redundant (same action and one rule’s match is a subset of the other).

DEFINITIONS & NORMALIZATION
- Floodlight drop = the "actions" key is absent. (Empty string is treated as drop ONLY if present in the input.)
- Actions are a comma-separated list in a single string (e.g., "set_queue=1,output=2"). Compare actions as an unordered set of tokens; ignore whitespace and token order.
- Consider only top-level match keys (e.g., "in_port", "eth_type", "ip_proto", "ipv4_src", "ipv4_dst", "tcp_dst", "udp_dst", "eth_vlan_vid", etc.). Keys like "name", "active", "priority" do not affect matching.
- Normalize values before comparison:
  • eth_type: "0x800" == "0x0800" (IPv4).  
  • ip_proto: accept "6"/"tcp", "17"/"udp", "1"/"icmp".  
  • ports and priorities: compare numerically even if strings.  
  • IPs may be host (/32) or CIDR; treat "10.0.0.1" as "10.0.0.1/32".
- Match overlap rules:
  • If "eth_type" differ (e.g., IPv4 vs IPv6), no overlap.  
  • If "ip_proto" differ (e.g., TCP vs UDP), no overlap.  
  • If both specify "in_port" with different values, no overlap.  
  • IP prefixes overlap if their CIDRs intersect (e.g., 10.0.0.0/24 overlaps 10.0.0.1/32).  
  • TCP/UDP port equality is required when both specify the same L4 field.  
  • VLAN: "eth_vlan_vid" must be equal if both specify it; otherwise the more general one (no VLAN constraint) overlaps the specific one.  
  • Absence of a field makes a rule more general on that dimension.

GUIDANCE
- Different "switch" → no conflict. If both specify "table" and they differ → no conflict.  
- Overlap + different actions → conflict.  
- Overlap + same actions:
    • If one match ⊂ the other → conflict (Redundancy).  
    • If partial overlap without subset and same actions → treat as no conflict for this binary decision.
- Priority does not affect the YES/NO decision, but may be used in the explanation (e.g., “more general rule may shadow a specific rule if higher priority”).

EXAMPLES

Example A: Conflict (Different Action)
F1: {"switch":"...:01","eth_type":"0x0800","ipv4_dst":"10.0.0.1","actions":"output=1"}
F2: {"switch":"...:01","eth_type":"0x0800","ipv4_dst":"10.0.0.1","actions":"output=2"}
Output: {"conflict_status":1,"conflict_explanation":"Same switch and identical match; actions differ (output=1 vs output=2)."}

Example B: No Conflict (Different Switch)
F1: {"switch":"...:01","eth_type":"0x0800","ipv4_dst":"10.0.0.1","actions":"output=3"}
F2: {"switch":"...:02","eth_type":"0x0800","ipv4_dst":"10.0.0.1","actions":"output=3"}
Output: {"conflict_status":0,"conflict_explanation":"Different switches."}

Example C: Conflict (Redundancy)
F1: {"switch":"...:04","eth_type":"0x0800","ip_proto":"6","tcp_dst":"80","actions":"output=2"}
F2: {"switch":"...:04","eth_type":"0x0800","ip_proto":"6","actions":"output=2"}
Output: {"conflict_status":1,"conflict_explanation":"Specific rule is subset of a more general rule with the same action (redundancy)."}

Example D: No Conflict (Protocol Mismatch)
F1: {"switch":"...:01","eth_type":"0x0800","ip_proto":"6","tcp_dst":"22","actions":"output=1"}
F2: {"switch":"...:01","eth_type":"0x0800","ip_proto":"17","udp_dst":"53","actions":"output=1"}
Output: {"conflict_status":0,"conflict_explanation":"TCP vs UDP are mutually exclusive."}

INPUT FORMAT
You will be provided with two JSON flow rules:

Flow 1:
<JSON>

Flow 2:
<JSON>

EXPECTED OUTPUT (strict JSON only):
{
  "conflict_status": <0 or 1>,
  "conflict_explanation": "<brief reason or empty string>"
}
NO EXTRA TEXT.
"""


In [4]:
# model names should match the downloaded ollama model names. 
# If ollama does not find a model, check ollama model list (write in terminal: "ollama list" without quote) and model names. Use the correct model name.

my_models=[
    "codegemma:7b",
    "codestral:22b",
    "codellama:34b",
    "codellama:7b",
    "command-r:35b",
    "deepseek-coder:1.3b",
    "Deepseek-coder-v2:16b",
    "dolphin-mistral:7b",
    "gemma2:27b",
    "huihui_ai/qwq-abliterated:latest",
    "huihui_ai/qwq-fusion:latest",
    "llama2:7b",
    "Llama3:8b",
    "llama3.1:8b",
    "llama3.2:3b",
    "llava-llama3:8b",
    "marco-o1:7b",
    "mistral:latest",
    "mistral-nemo:12b",
    "openchat:7b",
    "orca-mini:3b",
    "phi:2.7b",
    "phi3:3.8b",
    "qwen:4b",
    "qwen2:7b",
    "qwen2.5:7b",
    "qwq:latest",
    "starcoder:3b",
    "starcoder2:3b",
    "TinyLlama:1.1b",
    "wizardlm2:7b",
    "yi:6b",
    "zephyr:7b"
]

very_large_models = [
"llama3.3:latest",
"llama2:70b",
"codellama:70b"]

default_model = "llama2:7b"

#replace with your host IP or use localhost. Make sure the correct port number of ollama server.
ollama_embedding_url = "http://localhost:11434"
ollama_server_url = "http://localhost:11435"  

ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

In [5]:
def run_LLM_conflict(flow_json_1, flow_json_2, model):
    
    system_prompt = CONFLICT_PROMPT_FLOODLIGHT
    count = 0
    while True:
        count+=1
        try:
            time.sleep(0.1)

            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                stream=False,
                system=system_prompt,
                prompt=f"Flow 1:\n{json.dumps(flow_json_1, indent=2)}\n\nFlow 2:\n{json.dumps(flow_json_2, indent=2)}",
                format='json'
            )

            output = response['response'].strip()

            response_json = json.loads(output)

            if 'conflict_status' not in response_json:
                #print("\nWarning: 'conflict_status' key is missing in the response.\n")
                break
            else:
                valid_conflict_response = True
                conflict_status = response_json.get('conflict_status', 0)
                # Ensure conflict_status is an integer
                if isinstance(conflict_status, str):
                    conflict_status = int(conflict_status)

                return valid_conflict_response, conflict_status, response_json['conflict_explanation']             

        except Exception as e:
            print("Exception found for model: ",model, ". Reason: ", e)
            sys.stdout.flush()
            if(count<15):
                continue
            else:
                print("\n",model, " failed to produce valid JSON for conflict info after 15 tries. Going to next model\n")
                break               
    
    return False, None, None

In [6]:
def safe_load_flow_json(flow_str, index_number):
    try:
        return ast.literal_eval(flow_str)
    except Exception as e:
        print(f"[Index {index_number}] Error parsing flow JSON from excel using ast.literal_eval: {e}")
        return None


In [ ]:
df = pd.read_excel("FlowConflict-Floodlight.xlsx")

summary_rows = []

for model in my_models:
    TP = FP = TN = FN = 0
    times = []

    for idx, row in df.iterrows():

        index_number = row["Sl."]
        flow_rule_1_json = safe_load_flow_json(row["Floodlight Flow Rule 1"], index_number)
        flow_rule_2_json = safe_load_flow_json(row["Floodlight Flow Rule 2"], index_number)

        ground_truth_conflict = row["Conflicting"].strip().lower()  # 'yes' or 'no'
        gt_conflict = (ground_truth_conflict == 'yes')

        if (flow_rule_1_json is None) or (flow_rule_2_json is None):
            continue

        start_conflict_time = time.time()
        valid_conflict_response, conflict_status, conflict_details = run_LLM_conflict(
            flow_rule_1_json, flow_rule_2_json, model=model)
        conflict_duration = time.time() - start_conflict_time
        times.append(conflict_duration)

        # Only score if a valid result was returned
        if conflict_status == 2 or not valid_conflict_response:
            continue
        detected_conflict = (conflict_status == 1)

        # Update counts
        if detected_conflict and gt_conflict:
            TP += 1
        elif detected_conflict and not gt_conflict:
            FP += 1
        elif not detected_conflict and not gt_conflict:
            TN += 1
        elif not detected_conflict and gt_conflict:
            FN += 1

    total = TP + TN + FP + FN
    accuracy = (TP + TN) / total if total > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    fpr = FP / (FP + TN) if (FP + TN) > 0 else 0
    avg_time = sum(times) / len(times) if times else 0

    summary_rows.append([
        model, TP, FP, TN, FN, round(accuracy, 4), round(precision, 4),
        round(recall, 4), round(f1, 4), round(fpr, 4), round(avg_time, 4)
    ])

# ---- Write summary CSV ----
with open("Floodlight_conflict_detection_model_summaries.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Model", "TP", "FP", "TN", "FN", "Accuracy", "Precision", "Recall", "F1", "FPR", "Avg Time (s)"])
    for row in summary_rows:
        writer.writerow(row)

print("Processing completed. Model summaries saved to CSV.")